# Monte Carlo Tree Search (MCTS)

Monte Carlo Tree Search is an optimization technique used in decision-making processes, especially in games with large state spaces (like Go or Chess). It balances exploration (trying out new actions) and exploitation (focusing on actions that have yielded good results).

## Key Components

1. **Tree Structure**:
   - Each node represents a state of the game.
   - Edges represent actions leading to child states.

2. **MCTS Phases**:
   1. **Selection**: 
      Traverse the tree from the root to a leaf node by selecting child nodes according to a strategy, typically based on the Upper Confidence Bound for Trees (UCB1).

      $ UCB1 = \arg\max_{a \in A(s)} \left( \overline{Q}(s, a) + c \sqrt{\frac{\ln(N(s))}{N(s, a)}} \right) $

      where:
      - $ \overline{Q}(s, a) $ is the average reward for action $ a $ in state $ s $,
      - $ N(s) $ is the number of times the state $ s $ has been visited,
      - $ N(s, a) $ is the number of times action $ a $ has been taken in state $ s $,
      - $ c $ is the exploration constant.

   2. **Expansion**: 
      If the selected node isn't terminal, expand it by adding one or more child nodes corresponding to possible actions.

   3. **Simulation**: 
      Run a Monte Carlo simulation from the new node until reaching a terminal state. This involves randomly playing out the game to evaluate the result.

   4. **Backpropagation**: 
      Backpropagate the result of the simulation up the tree, updating statistics (e.g., number of visits and rewards) for the nodes along the path.

## Updates

- **Selection (UCB Formula)**:
  The UCB formula balances exploration (nodes that haven't been visited much) and exploitation (nodes that have given good rewards).
  
  - **Exploitation**: $ \overline{Q}(s, a) $, which is the average reward for action $ a $.
  - **Exploration**: $ \sqrt{\frac{\ln(N(s))}{N(s, a)}} $, which encourages exploration of less visited nodes.

- **Backpropagation**:
  Let the reward from the simulation be $ R $. For each node $ s_i $ in the path:

  $ N(s_i) \leftarrow N(s_i) + 1 $
  
  $ Q(s_i) \leftarrow Q(s_i) + \frac{R - Q(s_i)}{N(s_i)} $

  This updates the visit count and adjusts the average reward based on the new result.

We'll do an example implementation of this for tic-tac-toe. [Link](https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1) to where I got this implementation from.

In [1]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math


class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1 - reward if invert_reward else reward
            node = node.find_random_child()
            invert_reward = not invert_reward

    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward  # 1 for me is 0 for my enemy, and vice versa

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(self.children[node], key=uct)


class Node(ABC):
    """
    A representation of a single board state.
    MCTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """

    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        return set()

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

    @abstractmethod
    def reward(self):
        "Assumes `self` is terminal node. 1=win, 0=loss, .5=tie, etc"
        return 0

    @abstractmethod
    def __hash__(self):
        "Nodes must be hashable"
        return 123456789

    @abstractmethod
    def __eq__(node1, node2):
        "Nodes must be comparable"
        return True

In [11]:
def _winning_combos():
    for start in range(0, 9, 3):  # three in a row
        yield (start, start + 1, start + 2)
    for start in range(3):  # three in a column
        yield (start, start + 3, start + 6)
    yield (0, 4, 8)  # down-right diagonal
    yield (2, 4, 6)  # down-left diagonal

def _find_winner(tup):
    "Returns None if no winner, True if X wins, False if O wins"
    for i1, i2, i3 in _winning_combos():
        v1, v2, v3 = tup[i1], tup[i2], tup[i3]
        if False is v1 is v2 is v3:
            return False
        if True is v1 is v2 is v3:
            return True
    return None

In [8]:
from collections import namedtuple
from random import choice
import sys

_TTTB = namedtuple("TicTacToeBoard", "tup turn winner terminal")

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class TicTacToeBoard(_TTTB, Node):
    def find_children(board):
        if board.terminal:  # If the game is finished then no moves can be made
            return set()
        # Otherwise, you can make a move in each of the empty spots
        return {
            board.make_move(i) for i, value in enumerate(board.tup) if value is None
        }

    def find_random_child(board):
        if board.terminal:
            return None  # If the game is finished then no moves can be made
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    def reward(board):
        if not board.terminal:
            raise RuntimeError(f"reward called on nonterminal board {board}")
        if board.winner is board.turn:
            # It's your turn and you've already won. Should be impossible.
            raise RuntimeError(f"reward called on unreachable board {board}")
        if board.turn is (not board.winner):
            return 0  # Your opponent has just won. Bad.
        if board.winner is None:
            return 0.5  # Board is a tie
        # The winner is neither True, False, nor None
        raise RuntimeError(f"board has unknown winner type {board.winner}")

    def is_terminal(board):
        return board.terminal

    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1 :]
        turn = not board.turn
        winner = _find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    def to_pretty_string(board):
        to_char = lambda v: ("X" if v is True else ("O" if v is False else " "))
        rows = [
            [to_char(board.tup[3 * row + col]) for col in range(3)] for row in range(3)
        ]
        return (
            "\n  1 2 3\n"
            + "\n".join(str(i + 1) + " " + " ".join(row) for i, row in enumerate(rows))
            + "\n"
        )

In [10]:
def play_game():
    tree = MCTS()
    board = new_tic_tac_toe_board()
    print(board.to_pretty_string())
    while True:
        row_col = input("enter row,col: ")
        row, col = map(int, row_col.split(","))
        index = 3 * (row - 1) + (col - 1)
        if board.tup[index] is not None:
            raise RuntimeError("Invalid move")
        board = board.make_move(index)
        print(board.to_pretty_string())
        # flush stdout, so the above output is visible
        sys.stdout.flush()
        if board.terminal:
            break
        # You can train as you go, or only at the beginning.
        # Here, we train as we go, doing fifty rollouts each turn.
        for _ in range(50):
            tree.do_rollout(board)
        board = tree.choose(board)
        print(board.to_pretty_string())
        if board.terminal:
            break


def new_tic_tac_toe_board():
    return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)

In [9]:
play_game()


  1 2 3
1      
2      
3      


  1 2 3
1 X    
2      
3      


  1 2 3
1 X    
2   O  
3      


  1 2 3
1 X    
2 X O  
3      


  1 2 3
1 X    
2 X O  
3 O    


  1 2 3
1 X    
2 X O X
3 O    


  1 2 3
1 X   O
2 X O X
3 O    

